# EnigmaNet: A Neural Network Framework for Eplipesy Classification
### This is a script that uses Keras to predict epilepsy outcomes

## Script Details
This script is designed to run with the following configuration:
* Keras
* TensorFlow backend
* PlaidML backend (if using Radeon GPU)
* Tabular data (Excel or CSVs)

## Dependencies
Run the following command to pip install all dependencies
```
pip install numpy pandas sklearn matplotlib scipy keras_tqdm imblearn
```

Conda users can search for same packages and install them via `conda install <package_name>`

Also install NeuroCombat at https://github.com/ncullen93/neuroCombat

In [1]:
import enigma as en

Using TensorFlow backend.


## Create Enigma Object
Use various parameters to create the Enigmanet object

In [2]:
inData = en.enigmanet(path='/Users/sid/Documents/Projects/Enigma-ML/Dataset/T1/all.csv',
                 classcol='Dx',
                 sitecol='Site',
                 dbegin='ICV',
                 dend='R_insula_surfavg',
                 fillmissing=True,
                 harmonize=True,
                 crange=['Site', 'Sex'],
                 batch='Site',
                 discrete=['Dx', 'Sex'],
                 continuous=['Age'],
                 scale=True,
                 testsplit=0.30)

NameError: name 'en' is not defined

In [3]:
inData.transformdata()

...found 2 classes across 16 sites
...filling missing data with class means
Creating design matrix..
Standardizing data across features..
Fitting L/S model and finding priors..
Finding parametric adjustments..
Final adjustment of data..


In [4]:
x_train, x_test, y_train, y_test = inData.splitdata(oversample=True)
print('Dataset containing ' + str(len(inData.dframe.loc[:, inData.classcol])) + ' observations split into:')
print('    Training: ' + str(len(y_train)))
print('    Testing: ' + str(len(y_test)))

Dataset containing 2776 observations split into:
    Training: 2130
    Testing: 833
